### Replica 11: Replicar el zeroshot

#### A. Prueba de TransformerRouter

In [11]:
import pandas as pd
from haystack.components.routers import TransformersTextRouter

#! Seleccionar el modelo del router
text_router = TransformersTextRouter(model="shahrukhx01/bert-mini-finetune-question-detection")
text_router.warm_up()

#! Definir queries de prueba
queries = [
    "Arya Stark father",  # Keyword Query
    "Who was the father of Arya Stark",  # Interrogative Query
    "Lord Eddard was the father of Arya Stark",  # Statement Query
]

result = text_router.run(text=queries[0])
print(next(iter(result)))

#! Realizar las predicciones
results = {"Query": [], "Output Branch": [], "Class": []}
for query in queries:
    result = text_router.run(text=query)
    results["Query"].append(query)
    results["Output Branch"].append(next(iter(result)))
    results["Class"].append("Keyword Query" if next(iter(result)) == "LABEL_0" else "Question/Statement")

pd.DataFrame.from_dict(results)

Device set to use cuda:0


LABEL_0


/home/nahumfg/GithubProjects/TesisMaestria/TesisHaystack/venv_haystack_ai/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,Query,Output Branch,Class
0,Arya Stark father,LABEL_0,Keyword Query
1,Who was the father of Arya Stark,LABEL_1,Question/Statement
2,Lord Eddard was the father of Arya Stark,LABEL_1,Question/Statement


In [10]:
text_router = TransformersTextRouter(model="shahrukhx01/question-vs-statement-classifier")
text_router.warm_up()

queries = [
    "Who was the father of Arya Stark",  # Interrogative Query
    "Lord Eddard was the father of Arya Stark",  # Statement Query
]

results = {"Query": [], "Output Branch": [], "Class": []}
for query in queries:
    result = text_router.run(text=query)
    results["Query"].append(query)
    results["Output Branch"].append(next(iter(result)))
    results["Class"].append("Question" if next(iter(result)) == "LABEL_1" else "Statement")

pd.DataFrame.from_dict(results)

Device set to use cuda:0
/home/nahumfg/GithubProjects/TesisMaestria/TesisHaystack/venv_haystack_ai/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,Query,Output Branch,Class
0,Who was the father of Arya Stark,LABEL_1,Question
1,Lord Eddard was the father of Arya Stark,LABEL_0,Statement


#### B. Casos de uso personalizado para la clasificación de texto

In [17]:
text_router = TransformersTextRouter(model="cardiffnlp/twitter-roberta-base-sentiment")
text_router.warm_up()

queries = [
    "What's the answer?",  # neutral query
    "Would you be so lovely to tell me the answer?",  # positive query
    "Can you give me the damn right answer for once??",  # negative query
]

sent_results = {"Query": [], "Output Branch": [], "Class": []}
for query in queries:
    result = text_router.run(text=query)
    sent_results["Query"].append(query)
    sent_results["Output Branch"].append(next(iter(result)))
    sent_results["Class"].append({"LABEL_0": "negative", "LABEL_1": "neutral", "LABEL_2":"positive"}.get(next(iter(result)), "Unknown"))

pd.DataFrame.from_dict(sent_results)

Device set to use cuda:0
/home/nahumfg/GithubProjects/TesisMaestria/TesisHaystack/venv_haystack_ai/lib/python3.11/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


,Query,Output Branch,Class
0,What's the answer?,LABEL_1,neutral
1,Would you be so lovely to tell me the answer?,LABEL_2,positive
2,Can you give me the damn right answer for once??,LABEL_0,negative


#### C. Clasificación ZeroShot

In [24]:
import pandas as pd
from haystack.components.routers import TransformersZeroShotTextRouter

text_router = TransformersZeroShotTextRouter(labels=["music", "cinema"])
text_router.warm_up()

queries = [
    "In which films does John Travolta appear?",  # cinema
    "What is the Rolling Stones first album?",  # music
    "Who was Sergio Leone?",  # cinema
]


sent_results = {"Query": [], "Output Branch": []}
for query in queries:
    result = text_router.run(text=query)
    sent_results["Query"].append(query)
    sent_results["Output Branch"].append(next(iter(result)))

pd.DataFrame.from_dict(sent_results)


Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,Query,Output Branch
0,In which films does John Travolta appear?,cinema
1,What is the Rolling Stones first album?,music
2,Who was Sergio Leone?,cinema


In [25]:
from haystack.components.routers import TransformersZeroShotTextRouter

text_router = TransformersZeroShotTextRouter(labels=["Game of Thrones", "Star Wars", "Lord of the Rings"])
text_router.warm_up()

queries = [
    "Who was the father of Arya Stark",  # Game of Thrones
    "Who was the father of Luke Skywalker",  # Star Wars
    "Who was the father of Frodo Baggins",  # Lord of the Rings
]

results = {"Query": [], "Output Branch": []}

for query in queries:
    result = text_router.run(text=query)
    results["Query"].append(query)
    results["Output Branch"].append(next(iter(result)))

pd.DataFrame.from_dict(results)


Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,Query,Output Branch
0,Who was the father of Arya Stark,Game of Thrones
1,Who was the father of Luke Skywalker,Star Wars
2,Who was the father of Frodo Baggins,Lord of the Rings


In [27]:
import pandas as pd
from haystack.components.routers import TransformersZeroShotTextRouter

text_router = TransformersZeroShotTextRouter(labels=["general", "congreso", "contrataciones"])
text_router.warm_up()

queries = [
    "Hola",
    "Que puedes hacer",
    "Muestrame las asistencias del congreso de la republica",
    "Cuales son las empresas con más contrataciones",
]


sent_results = {"Query": [], "Output Branch": []}
for query in queries:
    result = text_router.run(text=query)
    sent_results["Query"].append(query)
    sent_results["Output Branch"].append(next(iter(result)))

pd.DataFrame.from_dict(sent_results)


Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,Query,Output Branch
0,Hola,general
1,Que puedes hacer,general
2,Muestrame las asistencias del congreso de la r...,congreso
3,Cuales son las empresas con más contrataciones,contrataciones
